In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as web    #to get the stock prices
from datetime import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [3]:
start = datetime(2000, 1, 1)
end = datetime.now()

df = web.DataReader("AAPL", 'yahoo', start, end)
df = df.reset_index()
df = df[['Date', 'Close','Open','Low','High']]
df.describe()

Close         Open          Low         High
count  5332.000000  5332.000000  5332.000000  5332.000000
mean     19.731317    19.728394    19.508618    19.939620
std      25.956312    25.960731    25.622978    26.269164
min       0.234286     0.231964     0.227143     0.235536
25%       1.396831     1.391607     1.377500     1.416652
50%       9.356607     9.391250     9.283393     9.457143
75%      28.013750    28.005624    27.666251    28.258750
max     143.160004   143.600006   141.369995   145.089996

In [4]:
index = int(df.shape[0]*0.6)
train_set = df.head(index)
dataClosed = train_set.iloc[:, 1:2].values
train_set

Date      Close       Open        Low       High
0    2000-01-03   0.999442   0.936384   0.907924   1.004464
1    2000-01-04   0.915179   0.966518   0.903460   0.987723
2    2000-01-05   0.928571   0.926339   0.919643   0.987165
3    2000-01-06   0.848214   0.947545   0.848214   0.955357
4    2000-01-07   0.888393   0.861607   0.852679   0.901786
...         ...        ...        ...        ...        ...
3194 2012-09-12  23.921070  23.816071  23.428572  23.924999
3195 2012-09-13  24.392143  24.191786  24.098928  24.482143
3196 2012-09-14  24.688572  24.641430  24.567499  24.892143
3197 2012-09-17  24.992144  24.976786  24.807501  24.992857
3198 2012-09-18  25.068214  24.995714  24.872143  25.083214

[3199 rows x 5 columns]

In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
dataClosedScaled = sc.fit_transform(dataClosed)
len(dataClosedScaled)

3199

In [6]:
x_train = []
y_train = []
for i in range(60, len(dataClosedScaled)):
    x_train.append(dataClosedScaled[i-60:i, 0])
    y_train.append(dataClosedScaled[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
len(x_train)

3139

In [7]:
from keras.models import Sequential  
from keras.layers import Dense        
from keras.layers import LSTM        
from keras.layers import Dropout      

In [ ]:
model = Sequential()

model.add( LSTM (units = 512, return_sequences = True , input_shape = ( x_train.shape[1],1 ) ,recurrent_activation='sigmoid') )
model.add(Dropout(0.2))

model.add( LSTM (units = 256, return_sequences = True ) )
model.add(Dropout(0.2))

model.add( LSTM (units = 128, return_sequences = True ) )
model.add(Dropout(0.2))

model.add( LSTM (units = 512) )
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

history = model.fit(x_train, y_train, epochs = 75, batch_size = 64)

Epoch 1/75
50/50 [==============================] - 2s 42ms/step - loss: 0.0365
Epoch 2/75
50/50 [==============================] - 2s 41ms/step - loss: 6.3252e-04
Epoch 3/75
50/50 [==============================] - 2s 41ms/step - loss: 5.9085e-04
Epoch 4/75
50/50 [==============================] - 2s 41ms/step - loss: 5.4339e-04
Epoch 5/75
50/50 [==============================] - 2s 41ms/step - loss: 5.5918e-04
Epoch 6/75
50/50 [==============================] - 2s 41ms/step - loss: 5.1703e-04
Epoch 7/75
50/50 [==============================] - 2s 41ms/step - loss: 4.6990e-04
Epoch 8/75
50/50 [==============================] - 2s 41ms/step - loss: 5.5045e-04
Epoch 9/75
50/50 [==============================] - 2s 41ms/step - loss: 7.7981e-04
Epoch 10/75
50/50 [==============================] - 2s 41ms/step - loss: 5.7067e-04
Epoch 11/75
50/50 [==============================] - 2s 41ms/step - loss: 3.8297e-04
Epoch 12/75
50/50 [==============================] - 2s 41ms/step - loss: 4.06

In [ ]:
loss_train = history.history['loss']
plt.plot(loss_train, 'g', label='Training loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

mean = np.mean(loss_train)
mean

In [ ]:
test_set = df.tail(df.shape[0]-index)
real_values = test_set.iloc[:, 1:2].values

total_data = pd.concat(( train_set['Close'],test_set['Close']), axis = 0)

In [ ]:
inputs = total_data[len(total_data)-len(test_set)-60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60,len(real_values)):
    X_test.append(inputs[i-60:i,0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
predictions = model.predict(X_test)
predictions = sc.inverse_transform(predictions)

In [ ]:
plt.figure(figsize=(18,19))
plt.plot(real_values, color = 'black', label = 'Apple Stock Price')
plt.plot(predictions, color = 'green', label = 'Predicted Apple Stock Price')
plt.xticks(range(0,len(real_values),200),df['Date'].loc[::500],rotation=45)
plt.title('Apple Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Apple Stock Price')
plt.legend()
plt.show()